In [13]:
import pandas as pd
import numpy as np
df_players = pd.read_csv('c:\\Users\\johnc\\Documents\\Finance and Structures\\nba-chokes-testing\\data\\player_scoring_data.csv')
df_games = pd.read_csv('c:\\Users\\johnc\\Documents\\Finance and Structures\\nba-chokes-testing\\data\\choke_list_full.csv')
df_pbp = pd.read_csv('c:\\Users\\johnc\\Documents\\Finance and Structures\\nba-chokes-testing\\data\\pbp_data_full.csv')

In [2]:
df_players = df_players.set_index("game_id")
df_players.head()

,player,points_bf,points_af,team
game_id,,,,
21900837.0,Collins,24.0,11.0,home
21900837.0,Young,12.0,13.0,home
21900837.0,Reddish,18.0,2.0,home
21900837.0,Graham,5.0,0.0,home
21900837.0,Goodwin,2.0,0.0,home


In [3]:
df_games['game_id'] = pd.to_numeric(df_games.game_id)
df_games['matchup'] = df_games.home_team + " vs " + df_games.away_team
df_games = df_games.set_index('game_id')
df_games.head()

,season,home_team,away_team,home_choke,matchup
game_id,,,,,
21900837,2019-20,ATL,DAL,0.0,ATL vs DAL
21900656,2019-20,ATL,LAC,0.0,ATL vs LAC
21900624,2019-20,SAS,ATL,1.0,SAS vs ATL
21900491,2019-20,ORL,ATL,1.0,ORL vs ATL
21900431,2019-20,BKN,ATL,0.0,BKN vs ATL


In [4]:
df_tot = df_players.join(df_games).reset_index()
df_tot.head()

,game_id,player,points_bf,points_af,team,season,home_team,away_team,home_choke,matchup
0,21700002.0,Pachulia,6.0,0.0,home,2017-18,GSW,HOU,1.0,GSW vs HOU
1,21700002.0,Thompson,13.0,3.0,home,2017-18,GSW,HOU,1.0,GSW vs HOU
2,21700002.0,Green,7.0,2.0,home,2017-18,GSW,HOU,1.0,GSW vs HOU
3,21700002.0,Curry,13.0,9.0,home,2017-18,GSW,HOU,1.0,GSW vs HOU
4,21700002.0,McCaw,2.0,2.0,home,2017-18,GSW,HOU,1.0,GSW vs HOU


In [5]:
df_tot.loc[df_tot.team == "home", "team"] = df_tot.home_team
df_tot.loc[df_tot.team == "visitor", "team"] = df_tot.away_team
df_tot = df_tot[["game_id", "matchup", "player", "points_bf", "points_af", "team", "season"]]
df_tot.head()

,game_id,matchup,player,points_bf,points_af,team,season
0,21700002.0,GSW vs HOU,Pachulia,6.0,0.0,GSW,2017-18
1,21700002.0,GSW vs HOU,Thompson,13.0,3.0,GSW,2017-18
2,21700002.0,GSW vs HOU,Green,7.0,2.0,GSW,2017-18
3,21700002.0,GSW vs HOU,Curry,13.0,9.0,GSW,2017-18
4,21700002.0,GSW vs HOU,McCaw,2.0,2.0,GSW,2017-18


In [6]:
df_tot = df_tot[df_tot.game_id > 40000000]
df_tot.shape

(493, 7)

In [7]:
df_tot = pd.melt(df_tot, id_vars=["game_id", "matchup", "player", "team", "season"],
               value_vars=["points_bf", "points_af"],
               var_name="time", value_name="points")

In [8]:
df_tot["time"] = np.select([(df_tot["time"] == "points_bf"), (df_tot["time"] == "points_af")],
                           ["Before Choke/Comeback", "After Choke/Comeback"])
df_tot = df_tot.rename(columns={
    "game_id": "Game ID",
    "matchup": "Matchup",
    "player": "Player",
    "team": "Team",
    "season": "Season",
    "time": "Time",
    "points": "Points"})

In [9]:
d1 = dict()
for game_id in list(np.unique(df_tot["Game ID"])):
    matchup = df_tot.loc[df_tot["Game ID"] == game_id, "Matchup"].iloc[0]
    if matchup not in d1.keys():
        d1[matchup] = df_tot[df_tot["Game ID"] == game_id][["Player", "Team", "Season", "Time", "Points"]].to_dict('records')
    else:
        d1[matchup + ' 2'] = df_tot[df_tot["Game ID"] == game_id][["Player", "Team", "Season", "Time", "Points"]].to_dict('records')   

In [11]:
import json
with open('interactive_data.json', 'w') as out:
    json.dump(d1, out, indent=4)

### Creating new data format for D3.js

Original structure didn't work correctly

In [37]:
## shorten dataframes to playoff only games
df_pbp = df_pbp[df_pbp.game_id > 40000000]
df_players = df_players[df_players.game_id > 40000000]
df_games = df_games[df_games.game_id > 40000000]
df_pbp.loc[(df_pbp.home_pct == 0.62946), 'event']
df_players.head()

,player,points_bf,points_af,team,game_id
1906,Walker,14.0,9.0,home,41900302.0
1907,Theis,4.0,2.0,home,41900302.0
1908,Brown,8.0,13.0,home,41900302.0
1909,Tatum,9.0,12.0,home,41900302.0
1910,Kanter,9.0,0.0,home,41900302.0


In [51]:
# create dictionary with the information we need each game
d2 = dict()
for idx, row in df_games.iterrows():
    matchup = row.away_team + " @ " + row.home_team
    season = row.season
    game_id = row.game_id
    box_score = df_players[df_players.game_id == game_id]
    pbp = df_pbp[df_pbp.game_id == game_id]
    total_before = box_score.points_bf.sum()
    home_before = box_score[box_score.team == "home"].points_bf.sum()
    away_before = box_score[box_score.team == "visitor"].points_bf.sum()

    total_after = box_score.points_af.sum()
    home_after = box_score[box_score.team == "home"].points_af.sum()
    away_after = box_score[box_score.team == "visitor"].points_af.sum()

    #locate top 3 scorers on each side before infelction
    home_top3 = list(box_score[box_score.team == "home"].nlargest(3, 'points_bf')["player"])

    away_top3 = list(box_score[box_score.team == "visitor"].nlargest(3, 'points_bf')["player"])

    #find home percentages
    top3_home_pts_before = 0
    top3_home_pts_after = 0
    for home_player in home_top3:
        top3_home_pts_before += box_score.loc[(box_score.player == home_player), 'points_bf'].iloc[0]
        top3_home_pts_after += box_score.loc[(box_score.player == home_player), 'points_af'].iloc[0]
    top3_home_before = np.around(top3_home_pts_before / total_before, 5)
    top3_home_after = np.around(top3_home_pts_after / total_after, 5)
    bottom_home_pts_before = np.around((home_before - top3_home_pts_before) / total_before, 5)
    bottom_home_pts_after = np.around((home_after - top3_home_pts_after) / total_after, 5)
    # print(top3_home_before, top3_home_after, bottom_home_pts_before, bottom_home_pts_after)
    #find away percentages
    top3_away_pts_before = 0
    top3_away_pts_after = 0
    for away_player in away_top3:
        top3_away_pts_before += box_score.loc[(box_score.player == away_player), 'points_bf'].iloc[0]
        top3_away_pts_after += box_score.loc[(box_score.player == away_player), 'points_af'].iloc[0]
    top3_away_before = np.around(top3_away_pts_before / total_before, 5)
    top3_away_after = np.around(top3_away_pts_after / total_before, 5)
    bottom_away_pts_before = np.around((away_before - top3_away_pts_before) / total_before, 5)
    bottom_away_pts_after = np.around((away_after - top3_away_pts_after) / total_after, 5)
    # find the inflection point
    num_events = pbp.loc[(pbp.event == pbp.event.max()), 'event'].iloc[0]
    if row.home_choke == 1:
        infelction_event = pbp.loc[(pbp.home_pct == pbp.home_pct.max()), 'event'].iloc[0]
        infelction_pct = np.around(infelction_event / num_events, 5)
    else:
        infelction_event = pbp.loc[(pbp.home_pct == pbp.home_pct.min()), 'event'].iloc[0]
        infelction_pct = np.around(infelction_event / num_events, 5)
    
    # create dictionaries with values
    before_dict = {
        "game_pct": infelction_pct,
        "season": season,
        "home_top3_pct": top3_home_before,
        "home_bottom_pct": bottom_home_pts_before,
        "away_top3_pct": top3_away_before,
        "away_bottom_pct": bottom_away_pts_before
        }
    after_dict = {
        "game_pct": np.around(1 - infelction_pct, 5),
        "season": season,
        "home_top3_pct": top3_home_after,
        "home_bottom_pct": bottom_home_pts_after,
        "away_top3_pct": top3_away_after,
        "away_bottom_pct": bottom_away_pts_after
    }

    # add game to dictionary of games
    if matchup not in d2.keys():
        d2[matchup] = [before_dict, after_dict]
    else:
        d2[matchup + ' 2'] = [before_dict, after_dict]

            

MIA @ BOS
MIA @ BOS
BOS @ TOR
LAC @ DAL
LAC @ DEN
DEN @ LAC
HOU @ OKC
MIL @ MIA
ATL @ PHI
PHI @ ATL
DEN @ POR
UTA @ LAC
MIL @ BKN
IND @ BOS
POR @ DEN
SAS @ DEN
GSW @ POR
GSW @ POR
POR @ GSW
LAC @ GSW
MIL @ TOR
TOR @ MIL
OKC @ POR
PHI @ BOS
CLE @ TOR
CLE @ IND
GSW @ HOU
HOU @ GSW
PHI @ MIA
UTA @ OKC


In [36]:
df_players.loc[(df_players.player == 'Curry'), 'points_af'].iloc[0]

11.0

In [52]:
import json
with open('stacked_data.json', 'w') as out:
    json.dump(d2, out, indent=4)

In [50]:
d2

{'MIA @ BOS': [{'game_pct': 0.47677,
   'season': '2019-20',
   'home_top3_pct': 0.32039,
   'home_bottom_pct': 0.26214,
   'away_top3_pct': 0.29126,
   'away_bottom_pct': 0.12621},
  {'game_pct': 0.52323,
   'season': '2019-20',
   'home_top3_pct': 0.24038,
   'home_bottom_pct': 0.15385,
   'away_top3_pct': 0.23301,
   'away_bottom_pct': 0.375}],
 'MIA @ BOS 2': [{'game_pct': 0.68374,
   'season': '2019-20',
   'home_top3_pct': 0.37013,
   'home_bottom_pct': 0.16883,
   'away_top3_pct': 0.29221,
   'away_bottom_pct': 0.16883},
  {'game_pct': 0.31626,
   'season': '2019-20',
   'home_top3_pct': 0.24,
   'home_bottom_pct': 0.14667,
   'away_top3_pct': 0.16883,
   'away_bottom_pct': 0.26667}],
 'BOS @ TOR': [{'game_pct': 0.69024,
   'season': '2019-20',
   'home_top3_pct': 0.33803,
   'home_bottom_pct': 0.21127,
   'away_top3_pct': 0.3662,
   'away_bottom_pct': 0.08451},
  {'game_pct': 0.30976000000000004,
   'season': '2019-20',
   'home_top3_pct': 0.14035,
   'home_bottom_pct': 0.22807